<a href="https://colab.research.google.com/github/yukinaga/automl/blob/main/section_3/01_nlp_corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutoMLによる「自然言語処理」 -コーパスの解析-
ライブラリPyCaretを使い、AutoMLによる「自然言語処理」を行います。  
クラウドファンディングによる融資に関する文章を解析し、返済の可能性を判定しましょう。  
まずは、PyCaretによる文章の解析から始めます。

## PyCaretのインストール
AutoMLをサポートするライブラリ、PyCaretをバージョンを指定してインストールします。

In [ ]:
!pip install numpy==1.21.4 numba==0.53
!pip install pycaret==2.3.10
!pip install pandas-profiling==3.1.0
!python -m spacy download en

## Google Colabの設定
Google Colab環境でPyCaretのインタラクティブな要素を表示するためには、以下のコードを実行する必要があります。

In [ ]:
from pycaret.utils import enable_colab

enable_colab()

## データセットの読み込み
今回は、「Kiva Microfunds」のデータを使用します。  
Kiva Microfundsは、クラウドファンディングにより個人が世界中の低所得の起業家や学生に融資することを可能にする非営利団体です。  
Kivaの各融資のデータには、性別や所在地などの情報に加え、借り手のストーリーも含まれています。  
今回は、自然言語処理によりこの借り手のストーリの文章を解析します。  
データセットには、6,818のサンプルが含まれています。  
データセットにおける各特徴の説明は以下の通りです。

* country: 借り手の国
* en: 借り手の個人的なストーリー
* gender: 性別 (M=男性, F=女性)
* loan_amount: 承認された融資の金額
* nonpayment: 貸し手のタイプ（Lender＝Kivaに登録した個人ユーザー、Partner＝Kivaと協力するマイクロファイナンス機関)
* sector: 借り手の業種
* status: 融資のステータス(1-債務不履行, 0-返済)
 
今回は、enの列のみを使用します。  

https://www.kiva.org/ 

In [ ]:
from pycaret.datasets import get_data

data = get_data("kiva")  # データの取得
data.shape  # データの形状

## 環境の設定
PyCaretの環境を設定します。  
pycaret.nlpのsetup関数はPyCaretの環境を初期化しますが、PyCaretの他の関数を実行する前に呼び出す必要があります。  

その際に、解析対象であるターゲット列を指定する必要があります。

In [ ]:
from pycaret.nlp import setup

clf = setup(data=data, target="en", session_id=123)  # 環境の初期化

## モデルの作成
models関数により、全ての使用可能な機械学習モデルを確認することができます。

In [ ]:
from pycaret.nlp import models

models()  # 機械学習モデルの一覧


「トピックモデル」とは、機械学習や自然言語処理において、文書の集合体に出現する抽象的な「トピック」を発見するための統計モデルの一種です。

create_model関数は、モデルの訓練と評価を行います。  
以下のコードでは、トピックモデルの1つLatent Dirichlet Allocation (LDA)のモデルを作成し、訓練します。  
num_topicsでトピックの数を指定しますが、デフォルトのトピック数は4となります。  

In [ ]:
from pycaret.nlp import create_model

lda = create_model("lda", num_topics=5, multi_core = True)  # LDAのモデルを作成し訓練

訓練済みモデルの概要を表示します。  

In [ ]:
print(lda)

各ハイパーパラメータの意味は、pycaret.nlpの公式ドキュメントに解説があります。  
https://pycaret.readthedocs.io/en/latest/api/nlp.html#pycaret.nlp.create_model

## ラベルの割当て
`assign_model`関数により、各サンプルが各トピックに所属する割合を計算することができます。  

In [ ]:
from pycaret.nlp import assign_model

lda_results = assign_model(lda)
lda_results.head()

## 評価
plot_model関数により、様々な角度からコーパス全体、もしくは特定のトピックを分析することができます。  
https://pycaret.readthedocs.io/en/latest/api/nlp.html#pycaret.nlp.plot_model

plot_model関数で特にパラメータを指定しない場合、各単語の出現数がグラフで表示されます。

In [ ]:
from pycaret.nlp import plot_model

plot_model()

plotに"bigram"を指定すると、連続した2単語の出現数がグラフで表示されます。

In [ ]:
plot_model(plot="bigram")

topic_numにトピック名を指定することで、特定のトピックを解析することができます。  
以下のコードは、"Topic 1"という名前のトピックの、単語の出現数をグラフで表示します。

In [ ]:
plot_model(lda, topic_num="Topic 1")

plotに"topic_distribution"を指定すると、各トッピックに分類されたサンプル数をグラフで確認することができます。  
各トピックの棒にカーソルを当てると、各トピックのキーワードが表示されます。

In [ ]:
plot_model(lda, plot="topic_distribution")

t-SNE（T-Distributed Stochastic Neighbor Embedding ）は高次元データを低次元に変換して可視化するための次元削減アルゴリズムです。  
以下のコードのようにplotに"tsne"を指定すると、t-SNEによるデータの可視化が行われます。  
デフォルトでは3次元への圧縮となります。  

In [ ]:
plot_model(lda, plot="tsne")